In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
!kaggle datasets list -s fruits


Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
!kaggle datasets download -d moltean/fruits

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 403, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
from zipfile import ZipFile
dataset='/content/fruits.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('The dataset is extracted')

BadZipFile: File is not a zip file

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.preprocessing import LabelEncoder
import os

In [ ]:
training_path = '/content/fruits-360-original-size/fruits-360-original-size/Training'

In [ ]:
testing_path = '/content/fruits-360-original-size/fruits-360-original-size/Test'

In [ ]:
train_datagen = ImageDataGenerator(rescale=1/255.)
test_datagen = ImageDataGenerator(rescale=1/255.)

In [ ]:
train_data = train_datagen.flow_from_directory(directory = training_path,
                                              batch_size=32,
                                              class_mode="sparse",
                                              target_size=(224,224),
                                              shuffle=True)

In [ ]:
test_data = train_datagen.flow_from_directory(directory = testing_path,
                                              batch_size=32,
                                              class_mode="sparse",
                                              target_size=(224,224),
                                              shuffle=True)

In [ ]:
classes_train = os.listdir(training_path)
classes_test = os.listdir(testing_path)

In [ ]:
encoder = LabelEncoder()
encoder.fit(classes_train)
encoder.fit(classes_test)

class_labels_train = encoder.transform(classes_train)
class_labels_test = encoder.transform(classes_test)

In [ ]:
print("Training data Classes")
for i in range(len(classes_train)):
    print(f"{classes_train[i]} -> {class_labels_train[i]}")

In [ ]:
print("Test data Classes")
for i in range(len(classes_test)):
    print(f"{classes_test[i]} -> {class_labels_test[i]}")

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3,3), input_shape= (224,224,3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D())

model.add(Conv2D(64, (3,3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D())

model.add(Conv2D(128, (3,3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D())

model.add(Conv2D(256, (3,3), activation = 'relu', padding = 'same'))
model.add(MaxPooling2D())

model.add(Flatten())

model.add(Dense(256, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(131, activation = 'softmax'))


model.compile(optimizer = 'adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

In [ ]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5)

history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=15,
    steps_per_epoch=len(train_data)/4,
    validation_steps=len(test_data)/2,
    callbacks=[early_stopping]
)

In [ ]:
model.evaluate(test_data)

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('loss')
plt.xlabel('epochs')
plt.legend(['Train','Val'], loc= 'upper left')
plt.show()

In [ ]:
plt.figure(figsize=(8,4))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['Train', 'Val'], loc='upper left')
plt.show()

In [ ]:
!pip install watermark

In [ ]:
%load_ext watermark

In [ ]:
%watermark -a "Ananya Viswanath" -u -d -v -m